# Topics – Easy Topic Modeling in Python

The text mining technique **Topic Modeling** has become a popular statistical method for clustering documents. This [Jupyter notebook](http://jupyter.org/) introduces a step-by-step workflow, basically containing data preprocessing, the actual topic modeling using **latent Dirichlet allocation** (LDA), which learns the relationships between words, topics, and documents, as well as some interactive visualizations to explore the model.

LDA, introduced in the context of text analysis in [2003](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf), is an instance of a more general class of models called **mixed-membership models**. Involving a number of distributions and parameters, the topic model is typically performed using [Gibbs sampling](https://en.wikipedia.org/wiki/Gibbs_sampling) with conjugate priors and is purely based on word frequencies.

There have been written numerous introductions to topic modeling for humanists (e.g. [this one](http://scottbot.net/topic-modeling-for-humanists-a-guided-tour/)), which provide another level of detail regarding its technical and epistemic properties.

For this workflow, you will need a corpus (a set of texts) as plain text (`.txt`) or [TEI XML](http://www.tei-c.org/index.xml) (`.xml`). Using the `dariah_topics` package, you also have the ability to process the output of [DARIAH-DKPro-Wrapper](https://github.com/DARIAH-DE/DARIAH-DKPro-Wrapper), a command-line tool for *natural language processing*.

Topic modeling works best with very large corpora. The [TextGrid Repository](https://textgridrep.org/) is a great place to start searching for text data. Anyway, to demonstrate the technique, we provide one small text collection in the folder `grenzboten_sample` containing 15 diary excerpts, as well as 15 war diary excerpts, which appeared in *Die Grenzboten*, a German newspaper of the late 19th and early 20th century.

**Of course, you can work with your own corpus in this notebook.**

We're relying on the LDA implementation by [Allen B. Riddell](https://www.ariddell.org/), called [lda](http://pythonhosted.org/lda/index.html), which is very lightweight. Aside from that, we provide two more Jupyter notebooks:

* [IntroducingMallet](IntroducingMallet.ipynb), using LDA by [MALLET](http://mallet.cs.umass.edu/topics.php), which is known to be very robust. 
* [IntroducingGensim](IntroducingGensim.ipynb), using LDA by called [Gensim](https://radimrehurek.com/project/gensim/), which is attractive because of its multi-core support.

For more information in general, have a look at the [documentation](http://dev.digital-humanities.de/ci/job/DARIAH-Topics/doclinks/1/).

## First step: Installing dependencies

To work within this Jupyter notebook, you will have to import the `dariah_topics` library. As you do, `dariah_topics` also imports a couple of external libraries, which have to be installed first. `pip` is the preferred installer program in Python. Starting with Python 3.4, it is included by default with the Python binary installers. If you are interested in `pip`, have a look at [this website](https://docs.python.org/3/installing/index.html).

To install the `dariah_topics` library with all dependencies, open your commandline, go with `cd` to the folder `Topics` and run:

```
pip install -r requirements.txt
```

Alternatively, you can do:

```
python setup.py install
```

If you get any errors or are not able to install *all* dependencies properly, try [Stack Overflow](https://stackoverflow.com/questions/tagged/pip) for troubleshooting or create a new issue on our [GitHub page](https://github.com/DARIAH-DE/Topics).

**Important**: If you are on macOS or Linux, you will have to use `pip3` and `python3`.

### Some final words
As you probably already know, code has to be written in the grey cells. You execute a cell by clicking the **Run**-button (or **Ctrl + Enter**). If you want to run all cells of the notebook at once, click **Cell > Run All** or **Kernel > Restart & Run All** respectively, if you want to restart the Python kernel first. On the left side of an (unexecuted) cell stands `In [ ]:`. The empty bracket means, that the cell hasn't been executed yet. By clicking **Run**, a star appears in the brackets (`In [*]:`), which means the process is running. In most cases, you won't see that star, because your computer is faster than your eyes. You can execute only one cell at once, all following executions will be in the waiting line. If the process of a cell is done, a number appears in the brackets (`In [1]:`).

## Starting with topic modeling!

Execute the following cell to import modules from the `dariah_topics` library.

In [38]:
from cophi_toolbox import preprocessing
from dariah_topics import postprocessing
from dariah_topics import visualization

In [33]:
help(postprocessing)

Help on module dariah_topics.postprocessing in dariah_topics:

NAME
    dariah_topics.postprocessing

DESCRIPTION
    Postprocessing Text Data, Saving Matrices, Corpora and LDA Models
    *****************************************************************
    
    Functions of this module are for **postprocessing purpose**. You can save `document-term matrices <https://en.wikipedia.org/wiki/Document-term_matrix>`_, `tokenized corpora <https://en.wikipedia.org/wiki/Tokenization_(lexical_analysis)>`_ and `LDA models <https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation>`_, access topics, topic probabilites for documents, and word probabilities for each topic. All matrix variants provided in :func:`preprocessing.create_document_term_matrix()`_ are supported, as well as `lda <https://pypi.python.org/pypi/lda>`_, `Gensim <https://radimrehurek.com/gensim/>`_ and `MALLET <http://mallet.cs.umass.edu/topics.php>`_ models or output, respectively. Recurrent variable names are based on the foll

In [32]:
help(preprocessing)

Help on module cophi_toolbox.preprocessing in cophi_toolbox:

NAME
    cophi_toolbox.preprocessing

DESCRIPTION
    Preprocessing Text Data, Creating Matrices and Cleaning Corpora
    ***************************************************************
    
    Functions of this module are for **preprocessing purpose**. You can read text files, `tokenize <https://en.wikipedia.org/wiki/Tokenization_(lexical_analysis)>`_ and segment documents (if a document is chunked into smaller segments, each segment counts as one document), create and read `document-term matrices <https://en.wikipedia.org/wiki/Document-term_matrix>`_, determine and remove features. Recurrent variable names are based on the following conventions:
    
        * ``corpus`` means an iterable containing at least one ``document``.
        * ``document`` means one single string containing all characters of a text     file, including whitespaces, punctuations, numbers, etc.
        * ``dkpro_document`` means a pandas DataFrame c

Furthermore, we will need some additional functions from external libraries.

In [39]:
import metadata_toolbox.utils as metadata
import pandas as pd
from pathlib import Path
import lda

Let's not pay heed to any warnings right now and execute the following cell.

In [40]:
import warnings
warnings.filterwarnings('ignore')

## 1. Preprocessing

### 1.2. Reading a corpus of documents

#### Defining the path to the corpus folder

In the present example code, we are using the 30 diary excerpts from the folder `grenzboten`. To use your own corpus, change the path accordingly.

In [41]:
path_to_corpus = Path('data', 'grenzboten_sample')

#### Specifying the pattern of filenames for metadata extraction

You have the ability to extract metadata from the filenames. For instance, if your textfiles look like:

```
goethe_1816_stella.txt
```

the pattern would look like this:

```
{author}_{year}_{title}
```

So, let's try this for the example corpus.

In [42]:
pattern = '{author}_{year}_{title}'

#### Accessing file paths and metadata
We begin by creating a list of all the documents in the folder specified above. That list will tell the function `preprocessing.read_files` (see below) which text documents to read. Furthermore, based on filenames we can create some metadata, e.g. author and title.

In [43]:
meta = pd.concat([metadata.fname2metadata(str(path), pattern=pattern) for path in path_to_corpus.glob('*.txt')])
meta[:5] # by adding '[:5]' to the variable, only the first 5 elements will be printed

,author,year,title
data\grenzboten_sample\Beck_1844_Tagebuch_56.txt,Beck,1844,Tagebuch_56
data\grenzboten_sample\Berto_1915_Kriegstagebuch_94.txt,Berto,1915,Kriegstagebuch_94
data\grenzboten_sample\Castelli_1846_Tagebuch_51.txt,Castelli,1846,Tagebuch_51
data\grenzboten_sample\Cleinom_1914_Kriegstagebuch_94.txt,Cleinom,1914,Kriegstagebuch_94
data\grenzboten_sample\Dix_1914_Kriegstagebuch_37.txt,Dix,1914,Kriegstagebuch_37


#### Read listed documents from folder

In [44]:
corpus = list(preprocessing.read_files(meta.index))
corpus[0][:255] # printing the first 255 characters of the first document

'Tagebuch von Karl Beck. Man spricht seit vierzehn Tagen von einem vollständigen Ministerwechsel und es circuliren im Publicum die verschiedensten Combinationen, wobei heute ganz andere Namen genannt werden, als gestern und morgen wieder andere, als heute.'

Your `corpus` contains as much elements (`documents`) as texts in your corpus are. Each element of `corpus` is a list containing exactly one element, the text itself as one single string including all whitespaces and punctuations:

```
[['This is the content of your first document.'],
 ['This is the content of your second document.'],
 ...
 ['This is the content of your last document.']]
```

### 1.3. Tokenize corpus
Now, your `documents` in `corpus` will be tokenized. Tokenization is the task of cutting a stream of characters into linguistic units, simply words or, more precisely, tokens. The tokenize function `dariah_topics` provides is a simple Unicode tokenizer. Depending on the corpus, it might be useful to use an external tokenizer function, or even develop your own, since its efficiency varies with language, epoch and text type.

In [45]:
tokenized_corpus = [list(preprocessing.tokenize(document)) for document in corpus]

At this point, each `document` is represented by a list of separate token strings. As above, have a look at the first document (which has the index `0` as Python starts counting at 0) and show its first 14 words/tokens (that have the indices `0:13` accordingly).

In [46]:
tokenized_corpus[0][0:13]

['tagebuch',
 'von',
 'karl',
 'beck',
 'man',
 'spricht',
 'seit',
 'vierzehn',
 'tagen',
 'von',
 'einem',
 'vollständigen',
 'ministerwechsel']

### 1.4. Create a document-term matrix

The LDA topic model is based on a [document-term matrix](https://en.wikipedia.org/wiki/Document-term_matrix) of the corpus. In a document-term matrix, rows correspond to documents and columns correspond to terms or tokens respectively. The values are token frequencies for each document.

In [47]:
document_term_matrix = preprocessing.create_document_term_matrix(tokenized_corpus, meta['title'])
document_term_matrix[:5]

,die,der,und,in,den,von,zu,das,des,nicht,...,weitlinge,weitschichtige,welker,welscher,werthschätzung,wesentlicher,wichtigeren,widerliche,widersetzlichen,gasfrage
Tagebuch_56,90.0,92.0,84.0,70.0,30.0,26.0,25.0,16.0,25.0,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kriegstagebuch_94,11.0,32.0,24.0,12.0,8.0,17.0,0.0,3.0,5.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Tagebuch_51,226.0,177.0,188.0,111.0,73.0,62.0,93.0,60.0,35.0,78.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kriegstagebuch_94,39.0,48.0,34.0,28.0,15.0,25.0,4.0,5.0,11.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kriegstagebuch_37,40.0,34.0,15.0,17.0,10.0,19.0,5.0,6.0,18.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 1.5. Feature removal

*Stopwords* (also known as *most frequent tokens*) and *hapax legomena* are harmful for LDA and have to be removed from the corpus or the document-term matrix respectively. In this example, the 50 most frequent tokens will be categorized as stopwords.

**Hint**: Be careful with removing most frequent tokens, you might remove tokens quite important for LDA. Anyway, to gain better results, it is highly recommended to use an external stopwords list.

In this notebook, we combine the 50 most frequent tokens, hapax legomena and an external stopwordslist.

#### List the 100 most frequent tokens

In [48]:
stopwords = preprocessing.list_mfw(document_term_matrix, most_frequent_tokens=100)

These are the five most frequent words:

In [49]:
stopwords[:5]

['die', 'der', 'und', 'in', 'den']

#### List hapax legomena

In [50]:
hapax_legomena = preprocessing.find_hapax_legomena(document_term_matrix)
print("Total number of types in corpus:", document_term_matrix.shape[1])
print("Total number of hapax legomena:", len(hapax_legomena))

Total number of types in corpus: 24451
Total number of hapax legomena: 19757


In [22]:
help(preprocessing.find_hapax_legomena)

Help on function find_hapax_legomena in module cophi_toolbox.preprocessing:

find_hapax_legomena(document_term_matrix, type_ids=None)
    Creates a list with hapax legommena.
    
    With this function you can determine *hapax legomena* for each document.     Use the function :func:`create_document_term_matrix()` to create a     document-term matrix.
    
    Args:
        document_term_matrix (pandas.DataFrame): A document-term matrix.
        type_ids (dict): A dictionary with types as key and identifiers as values.
            If ``document_term_matrix`` is designed for large corpora, you have
            to commit ``type_ids``, too.
    
    Returns:
        Hapax legomena in a list.
    
    Example:
        >>> document_labels = ['document']
        >>> tokenized_corpus = [['hapax', 'stopword', 'stopword']]
        >>> document_term_matrix = create_document_term_matrix(tokenized_corpus, document_labels)
        >>> find_hapax_legomena(document_term_matrix)
        ['hapax']
    

#### Optional: Use external stopwordlist

In [51]:
path_to_stopwordlist = Path('data', 'stopwords', 'de.txt')
external_stopwords = [line.strip() for line in path_to_stopwordlist.open('r', encoding='utf-8')]

#### Combine lists and remove content from `document_term_matrix`

In [52]:
features = stopwords + hapax_legomena + external_stopwords
document_term_matrix = preprocessing.remove_features(features, document_term_matrix=document_term_matrix)

Finally, this is how your clean corpus looks like now.

In [53]:
document_term_matrix[:5]

,franzosen,genommen,abgewiesen,südlich,berlin,lassen,geschütze,englische,deutschland,januar,...,schlechteste,dubatowka,palameix,verschlossene,eimer,schicksale,eilwagen,klippe,rennt,zuschrieb
Tagebuch_56,0.0,1.0,0.0,0.0,4.0,3.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kriegstagebuch_94,0.0,9.0,3.0,5.0,3.0,0.0,6.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Tagebuch_51,1.0,1.0,1.0,0.0,1.0,7.0,0.0,1.0,9.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
Kriegstagebuch_94,4.0,3.0,2.0,3.0,2.0,0.0,2.0,7.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kriegstagebuch_37,6.0,1.0,0.0,0.0,0.0,0.0,5.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2. Model creation

The actual topic modeling is done with external state-of-the-art LDA implementations. In this example, we are relying on the open-source toolkit **lda**.

### 2.1. Creating list of vocabulary

To translate numbers back into words after the model creation, you have to set up a list of all unique tokens in the corpus.

In [54]:
vocabulary = document_term_matrix.columns
vocabulary

Index(['franzosen', 'genommen', 'abgewiesen', 'südlich', 'berlin', 'lassen',
       'geschütze', 'englische', 'deutschland', 'januar',
       ...
       'schlechteste', 'dubatowka', 'palameix', 'verschlossene', 'eimer',
       'schicksale', 'eilwagen', 'klippe', 'rennt', 'zuschrieb'],
      dtype='object', length=4242)

### 2.2. Translate document-term matrix into an array

In this step, all values of your document-term matrix will be translated into an [array](https://en.wikipedia.org/wiki/Array_data_structure).

In [55]:
document_term_matrix_arr = document_term_matrix.values.astype(int)
document_term_matrix_arr

array([[ 0,  1,  0, ...,  0,  0,  0],
       [ 0,  9,  3, ...,  0,  0,  0],
       [ 1,  1,  1, ...,  0,  0,  0],
       ...,
       [ 6,  6, 19, ...,  0,  0,  0],
       [ 3,  2,  0, ...,  0,  0,  0],
       [ 1,  3,  0, ...,  0,  0,  0]])

### 2.3. Generate LDA model

We use the class `LDA` from the library `lda` (which is basically not the same, because Python is case sensitive) to generate a LDA topic model. To instance a `LDA` object, there have to be specified a couple of parameters.

But first, if you are curious about any library, module, class or function, try `help()`. This can be very useful, because (at least in a well documented library) explanations of use and parameters will be printed. We're interested in the class `LDA` of the library `lda`, so let's try:

```
help(lda.LDA)
```

This will print something like this (in fact even more):

```
Help on class LDA in module lda.lda:

class LDA(builtins.object)
 |  Latent Dirichlet allocation using collapsed Gibbs sampling
 |  
 |  Parameters
 |  ----------
 |  n_topics : int
 |      Number of topics
 |  
 |  n_iter : int, default 2000
 |      Number of sampling iterations
 |  
 |  alpha : float, default 0.1
 |      Dirichlet parameter for distribution over topics
 |  
 |  eta : float, default 0.01
 |      Dirichlet parameter for distribution over words
 |  
 |  random_state : int or RandomState, optional
 |      The generator used for the initial topics.
```

So, now you know how to define the number of topics and the number of sampling iterations as well. A higher number of iterations will probably yield a better model, but also increases processing time. `alpha`, `eta` and `random_state` are so-called *hyperparameters*. They influence the model's performance, so feel free to play around with them. In the present example, we will leave the default values. Furthermore, there exist various methods for hyperparameter optimization, e.g. gridsearch or Gaussian optimization.

**Warning: This step can take quite a while!** Meaning something between some seconds and some hours depending on corpus size and the number of iterations. Our example corpus should be done within a minute or two at `n_iter=1000`.

In [56]:
%%time

model = lda.LDA(n_topics=10, n_iter=1000)
model.fit(document_term_matrix_arr)

Wall time: 5.74 s


### 2.4. Create document-topic matrix

The generated model object can now be translated into a human-readable document-topic matrix (that is a actually a pandas DataFrame) that constitutes our principle exchange format for topic modeling results.

In [57]:
help(lda.LDA.fit)

Help on function fit in module lda.lda:

fit(self, X, y=None)
    Fit the model with X.
    
    Parameters
    ----------
    X: array-like, shape (n_samples, n_features)
        Training data, where n_samples in the number of samples
        and n_features is the number of features. Sparse matrix allowed.
    
    Returns
    -------
    self : object
        Returns the instance itself.



In [58]:
topics = postprocessing.show_topics(model=model, vocabulary=vocabulary)
topics

NameError: name 'model' is not defined

In [35]:
help(postprocessing.show_topics)

Help on function show_topics in module dariah_topics.postprocessing:

show_topics(model=None, vocabulary=None, topic_keys_file=None, num_keys=10)
    Shows topics of LDA model.
    
    With this function you can show all topics of a LDA model in a pandas DataFrame.     For each topic, the top ``num_keys`` keys will be considered. If you have a
    * `lda <https://pypi.python.org/pypi/lda>`_ model, you have to pass the model     as ``model`` and the document-term matrix vocabulary as ``vocabulary``.
    * `Gensim <https://radimrehurek.com/gensim/>`_ model, you have to pass only the model     as ``model``.
    * `MALLET <http://mallet.cs.umass.edu/topics.php>`_ based workflow, you have to    pass only the ``topic_keys_file``.
    
    Args:
        model (optional): lda or Gensim model.
        vocabulary (list, optional): Only for lda. The vocabulary of the 
            document-term matrix.
        topic_keys_file (str): Only for MALLET. Path to the topic keys file.
        num_keys (

## 3. Model visualization

Each topic has a certain probability for each document in the corpus (have a look at the cell below). This probability distributions are visualized in an interactive **heatmap** (the darker the color, the higher the probability) which displays the kind of information
                that is presumably most useful to literary scholars. Going beyond pure exploration, this visualization can be used to show thematic developments over a set of texts as well as a single text, akin to a dynamic topic model. What might become
                apparent here, is that some topics correlate highly with a specific author or group of authors, while other topics correlate highly with a specific text or group of texts.

In [27]:
document_topics = postprocessing.show_document_topics(model=model, topics=topics, document_labels=meta['title'])
document_topics

NameError: name 'model' is not defined

### 3.1. Distribution of topics

#### Distribution of topics over all documents

The distribution of topics over all documents can now be visualized in a heatmap.

In [28]:
from bokeh.io import output_notebook, show
output_notebook()
%matplotlib inline

Loading BokehJS ...

In [29]:
PlotDocumentTopics = visualization.PlotDocumentTopics(document_topics)
show(PlotDocumentTopics.interactive_heatmap(), notebook_handle=True)

NameError: name 'document_topics' is not defined

Or a static heatmap:

In [ ]:
static_heatmap = PlotDocumentTopics.static_heatmap()
static_heatmap.show()